## package import & environment setting
---

In [1]:
!cd ~/aiffel && git clone https://github.com/huggingface/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [2]:
!cd transformers && pip install -e .

/bin/bash: line 0: cd: transformers: No such file or directory


In [3]:
!/opt/conda/bin/python3.7 -m pip install --upgrade pip

     |████████████████████████████████| 1.7 MB 22.1 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.3.3
    Uninstalling pip-20.3.3:
      Successfully uninstalled pip-20.3.3


In [4]:
!pip install datasets

     |████████████████████████████████| 311 kB 26.6 MB/s            
     |████████████████████████████████| 133 kB 62.2 MB/s            
     |████████████████████████████████| 243 kB 51.4 MB/s            
     |████████████████████████████████| 112 kB 61.4 MB/s            
     |████████████████████████████████| 76 kB 10.2 MB/s            
     |████████████████████████████████| 67 kB 11.4 MB/s            
     |████████████████████████████████| 25.6 MB 51.0 MB/s            
     |████████████████████████████████| 40 kB 8.7 MB/s             
     |████████████████████████████████| 86 kB 10.6 MB/s            
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.56.0
    Uninstalling tqdm-4.56.0:
      Successfully uninstalled tqdm-4.56.0
  Attempting uninstall: packaging
    Found existing installation: packaging 20.8
    Uninstalling packaging-20.8:
      Successfully uninstalled packaging-20.8
  Attempting uninstall: dill
    Found existing installation: dill 0.3.3
 

In [5]:
!pip install huggingface-hub==0.1.0

     |████████████████████████████████| 59 kB 11.7 MB/s            
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.4.0
    Uninstalling huggingface-hub-0.4.0:
      Successfully uninstalled huggingface-hub-0.4.0


In [6]:
!cd ~/aiffel/transformers

In [7]:
!python examples/tensorflow/text-classification/run_glue.py \
	--model_name_or_path bert-base-cased \
	--task_name mrpc \
	--output_dir ./models/mrpc \
	--overwrite_output_dir \
	--do_train \
	--do_eval \
	--num_train_epochs 1 \
	--save_steps 20000

2022-01-24 00:33:02.322066: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Traceback (most recent call last):
  File "examples/tensorflow/text-classification/run_glue.py", line 39, in <module>
    from transformers.trainer_utils import get_last_checkpoint, is_main_process
ImportError: cannot import name 'get_last_checkpoint' from 'transformers.trainer_utils' (/opt/conda/lib/python3.7/site-packages/transformers/trainer_utils.py)


In [8]:
!pip install tensorflow-datasets -U

     |████████████████████████████████| 4.0 MB 23.7 MB/s            
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.2.0
    Uninstalling tensorflow-datasets-4.2.0:
      Successfully uninstalled tensorflow-datasets-4.2.0


In [25]:
import os
import numpy as np
from argparse import ArgumentParser
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoConfig
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification, AutoConfig
# from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification
# from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, AutoConfig

from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import warnings

In [2]:
def init_gpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs\n\n")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e, "\n\n")
        
init_gpu()

1 Physical GPUs, 1 Logical GPUs




## loading data
---

In [3]:
data, info = tfds.load('glue/mnli', with_info=True)
# info.splits['train'].num_examples

INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: glue/mnli/2.0.0
INFO:absl:Load dataset info from /tmp/tmpemtxst6etfds
INFO:absl:Generating dataset glue (/aiffel/tensorflow_datasets/glue/mnli/2.0.0)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

INFO:absl:Downloading https://dl.fbaipublicfiles.com/glue/data/MNLI.zip into /aiffel/tensorflow_datasets/downloads/dl.fbaipublicfiles.com_glue_MNLIdNe8cK2kTBCG0bqBz2JxwShRT2KfuO3NVIwROTnjtfI.zip.tmp.20d63f6cc1674cf18c555a5deafc013d...


...

INFO:absl:Done writing glue-test_matched.tfrecord. Number of examples: 9796 (shards: [9796])


Generating test_mismatched examples...:   0%|          | 0/9847 [00:00<?, ? examples/s]

Shuffling glue-test_mismatched.tfrecord...:   0%|          | 0/9847 [00:00<?, ? examples/s]

INFO:absl:Done writing glue-test_mismatched.tfrecord. Number of examples: 9847 (shards: [9847])
INFO:absl:Constructing tf.data.Dataset glue for split None, from /aiffel/tensorflow_datasets/glue/mnli/2.0.0


Dataset glue downloaded and prepared to /aiffel/tensorflow_datasets/glue/mnli/2.0.0. Subsequent calls will reuse this data.


In [4]:
info.splits['train'].num_examples

392702

In [5]:
data['train'].take(1)

<TakeDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [6]:
examples = data['train'].take(1)
for example in examples:
    hypothesis = example['hypothesis']
    idx = example['idx']
    label = example['label']
    premise = example['premise']
    print(hypothesis.numpy())
    print(idx.numpy())
    print(label.numpy())
    print(premise.numpy())

b'Meaningful partnerships with stakeholders is crucial.'
16399
1
b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'


## MNLI Processor
---

In [7]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

print("슝~")

슝~


In [8]:
class MnliProcessor(DataProcessor):
    """Processor for the MRPC data set (GLUE version)."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        warnings.filterwarnings("ignore", category=DeprecationWarning) 

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")
    
    def get_labels(self):
        """See base class."""
        return ["0", "1", "2"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = f"{set_type}-{line[0]}"
            text_a = line[8]
            text_b = line[9]
            label = None if set_type.startswith("test") else line[-1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples
    
    print("슝~")

슝~


In [9]:
processor = MnliProcessor()
examples = data['train'].take(1)

for example in examples:
    print('------원본데이터------')
    print(example)  
    example = processor.get_example_from_tensor_dict(example)
    print('------processor 가공데이터------')
    print(example)

------원본데이터------
{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Meaningful partnerships with stakeholders is crucial.'>, 'idx': <tf.Tensor: shape=(), dtype=int32, numpy=16399>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'>}
------processor 가공데이터------
InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeh

In [10]:
examples = (data['train'].take(1))
for example in examples:
    example = processor.get_example_from_tensor_dict(example)
    example = processor.tfds_map(example)
    print(example)

InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeholders is crucial.', label='1')


In [11]:
label_list = processor.get_labels()
label_list

['0', '1', '2']

In [12]:
label_map = {label: i for i, label in enumerate(label_list)}
label_map

{'0': 0, '1': 1, '2': 2}

## data to dataset
---

In [13]:
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid))
        print("features: %s" % features[i])

    return features

In [14]:
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [15]:
hyp = {'bat': 8, 'max':128}

In [19]:
# train 데이터셋
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=hyp['max'], processor=processor)
train_dataset_batch = train_dataset.shuffle(100).batch(hyp['bat']).repeat(2)

Using label list ['0', '1', '2']
*** Example ***
guid: 16399
features: InputFeatures(input_ids=[101, 1999, 5038, 1997, 2122, 13136, 1010, 1048, 11020, 2038, 2499, 29454, 29206, 14626, 2144, 2786, 2000, 16636, 1996, 10908, 1997, 1996, 2110, 4041, 6349, 1998, 2000, 5323, 15902, 13797, 2007, 22859, 6461, 2012, 6469, 2075, 1037, 2047, 25353, 14905, 10735, 2483, 2090, 1996, 2976, 10802, 1998, 15991, 1997, 3423, 2578, 4804, 1012, 102, 15902, 13797, 2007, 22859, 2003, 10232, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [20]:
# validation 데이터셋
validation_dataset = tf_glue_convert_examples_to_features(data['validation_matched'], tokenizer, max_length=hyp['max'], processor=processor)
validation_dataset_batch = validation_dataset.shuffle(100).batch(hyp['bat'])

Using label list ['0', '1', '2']
*** Example ***
guid: 6287
features: InputFeatures(input_ids=[101, 7910, 1011, 9616, 2821, 3398, 2035, 1996, 2111, 2005, 2157, 7910, 2166, 2030, 2242, 102, 3398, 7167, 1997, 2111, 2005, 1996, 2157, 2166, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [21]:
test_dataset = tf_glue_convert_examples_to_features(data['test_matched'], tokenizer, max_length=hyp['max'], processor=processor)
test_dataset_batch = test_dataset.shuffle(100).batch(hyp['bat'])

Using label list ['0', '1', '2']
*** Example ***
guid: 5398
features: InputFeatures(input_ids=[101, 2092, 2003, 2045, 1037, 2210, 3751, 10216, 2017, 2404, 1999, 2045, 102, 2003, 2045, 1037, 10216, 2008, 2017, 2404, 1999, 2045, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [27]:
test_mis_dataset = tf_glue_convert_examples_to_features(data['test_mismatched'], tokenizer, max_length=hyp['max'], processor=processor)
test_mis_dataset_batch = test_dataset.shuffle(100).batch(hyp['bat'])

Using label list ['0', '1', '2']
*** Example ***
guid: 3498
features: InputFeatures(input_ids=[101, 4312, 1010, 1045, 8813, 2008, 2795, 1012, 102, 2008, 2795, 2038, 2042, 1999, 2026, 2155, 2005, 8213, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1

## model training
---

In [21]:
num_classes = len(processor.get_labels())

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

print("슝~")

슝~


In [23]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(train_dataset_batch, epochs=5, steps_per_epoch=115, validation_data=validation_dataset_batch)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


115/115 [==============================] - ETA: 0s - loss: 1.0991 - acc: 0.3695

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


115/115 [==============================] - 242s 2s/step - loss: 1.0988 - acc: 0.3700 - val_loss: 0.9930 - val_acc: 0.5472
Epoch 2/5
115/115 [==============================] - 222s 2s/step - loss: 0.9546 - acc: 0.5444 - val_loss: 0.8596 - val_acc: 0.6050
Epoch 3/5
115/115 [==============================] - 221s 2s/step - loss: 0.8724 - acc: 0.5983 - val_loss: 0.8096 - val_acc: 0.6539
Epoch 4/5
115/115 [==============================] - 222s 2s/step - loss: 0.7897 - acc: 0.6307 - val_loss: 0.8039 - val_acc: 0.6555
Epoch 5/5
115/115 [==============================] - 222s 2s/step - loss: 0.7831 - acc: 0.6766 - val_loss: 0.7668 - val_acc: 0.6766


## evaluating result
---

In [28]:
matched_result = model.evaluate(test_dataset_batch )
mismatched_result = model.evaluate(test_mis_dataset_batch)


print(f"\nMatched Test data\tLoss: {matched_result[0]:.3f}\tAccuracy: {matched_result[1]:.3f}")
print(f"Mismatched Test data\tLoss: {mismatched_result[0]:.3f}\tAccuracy: {mismatched_result[1]:.3f}")

1225/1225 [==============================] - 175s 143ms/step - loss: 1.6312 - acc: 0.2997

Matched Test data	Loss: 1.631	Accuracy: 0.300
Mismatched Test data	Loss: 1.631	Accuracy: 0.300


In [29]:
test_predict = model.predict(validation_dataset_batch.take(1))
test_predict

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


TFSequenceClassifierOutput(loss=None, logits=array([[-0.8614442 ,  1.1199486 , -0.48410103],
       [ 1.730397  , -0.7742273 , -0.8406817 ],
       [ 1.9241418 , -0.9086794 , -0.9488362 ],
       [-0.17124885, -0.22501676, -0.44525176],
       [ 2.1003218 , -0.9237179 , -1.0159171 ],
       [-1.2296417 ,  0.31685883,  0.37708497],
       [-0.8970667 ,  0.938266  , -0.46593297],
       [ 1.398127  , -0.625782  , -1.048554  ]], dtype=float32), hidden_states=None, attentions=None)

bert의 최종 accuracy는 0.3이었습니다.

## another model
---
bert보다 개선된 모델이라고 하여 진행했습니다.\
data to dataset부터 training까지 필요한 부분만 진행했습니다.\
참조: https://sajjjadayobi.github.io/blog/nlp/jupyter/2021/01/23/Text-Classification-Albert.html

In [16]:
albert_tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

In [19]:
#train~test dataset
albert_train_dataset = tf_glue_convert_examples_to_features(data['train'], albert_tokenizer, max_length=hyp['max'], processor=processor)
albert_train_dataset_batch = albert_train_dataset.shuffle(100).batch(hyp['bat']).repeat(2)

albert_validation_dataset = tf_glue_convert_examples_to_features(data['validation_matched'], albert_tokenizer, max_length=hyp['max'], processor=processor)
albert_validation_dataset_batch = albert_validation_dataset.shuffle(100).batch(hyp['bat'])

albert_test_dataset = tf_glue_convert_examples_to_features(data['test_matched'], albert_tokenizer, max_length=hyp['max'], processor=processor)
albert_test_dataset_batch = albert_test_dataset.shuffle(100).batch(hyp['bat'])

albert_test_mis_dataset = tf_glue_convert_examples_to_features(data['test_mismatched'], albert_tokenizer, max_length=hyp['max'], processor=processor)
albert_test_mis_dataset_batch = albert_test_dataset.shuffle(100).batch(hyp['bat'])

Using label list ['0', '1', '2']
*** Example ***
guid: 16399
features: InputFeatures(input_ids=[2, 19, 3514, 16, 158, 14978, 15, 644, 3862, 63, 577, 29153, 102, 179, 1485, 20, 11266, 14, 11870, 16, 14, 146, 2334, 5153, 17, 20, 4088, 16912, 16169, 29, 24757, 5414, 35, 4718, 68, 21, 78, 13, 7261, 2161, 8076, 128, 14, 1058, 11747, 17, 18560, 16, 1517, 687, 3234, 9, 3, 16912, 16169, 29, 24757, 25, 10421, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Using label list ['0', '1', '2']
*** Example ***
guid: 5398
features: InputFeatures(input_ids=[2, 134, 25, 80, 21, 265, 2098, 9786, 42, 442, 19, 80, 3, 25, 80, 21, 9786, 30, 42, 442, 19, 80, 60, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [26]:
albert_model = TFAlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=num_classes)

Downloading:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFAlbertForSequenceClassification.

Some layers of TFAlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
albert_model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
albert_model.summary()

Model: "tf_albert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
albert (TFAlbertMainLayer)   multiple                  11683584  
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 11,685,891
Trainable params: 11,685,891
Non-trainable params: 0
_________________________________________________________________


In [28]:
albert_model.fit(albert_train_dataset_batch, epochs=5, steps_per_epoch=115, validation_data=albert_validation_dataset_batch)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


115/115 [==============================] - ETA: 0s - loss: 1.1043 - acc: 0.3871

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


115/115 [==============================] - 287s 2s/step - loss: 1.1039 - acc: 0.3874 - val_loss: 0.9859 - val_acc: 0.4916
Epoch 2/5
115/115 [==============================] - 261s 2s/step - loss: 0.9616 - acc: 0.5433 - val_loss: 1.0065 - val_acc: 0.5045
Epoch 3/5
115/115 [==============================] - 259s 2s/step - loss: 0.9783 - acc: 0.5136 - val_loss: 0.8943 - val_acc: 0.6097
Epoch 4/5
115/115 [==============================] - 259s 2s/step - loss: 0.8550 - acc: 0.6273 - val_loss: 0.9006 - val_acc: 0.5897
Epoch 5/5
115/115 [==============================] - 259s 2s/step - loss: 0.8780 - acc: 0.6100 - val_loss: 0.8202 - val_acc: 0.6477


## evaluating result for albert
---

In [29]:
albert_matched_result = albert_model.evaluate(albert_test_dataset_batch )
albert_mismatched_result = albert_model.evaluate(albert_test_mis_dataset_batch)


print(f"\nMatched Test data\tLoss: {albert_matched_result[0]:.3f}\tAccuracy: {albert_matched_result[1]:.3f}")
print(f"Mismatched Test data\tLoss: {albert_mismatched_result[0]:.3f}\tAccuracy: {albert_mismatched_result[1]:.3f}")

1225/1225 [==============================] - 211s 172ms/step - loss: 1.2824 - acc: 0.3020

Matched Test data	Loss: 1.282	Accuracy: 0.302
Mismatched Test data	Loss: 1.282	Accuracy: 0.302


In [30]:
albert_test_predict = albert_model.predict(albert_validation_dataset_batch.take(1))
albert_test_predict

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


TFSequenceClassifierOutput(loss=None, logits=array([[ 1.3007646 , -0.6288863 , -0.4183692 ],
       [ 0.90966445, -0.20313501, -0.59984326],
       [-1.2203639 ,  0.6796348 ,  0.11481518],
       [-0.6809327 ,  0.30544478,  0.07592443],
       [-1.1741643 , -0.19257626,  0.96680737],
       [ 1.2521169 , -0.8349936 , -0.2649039 ],
       [-0.9011495 ,  0.92537045, -0.17351839],
       [-0.8458379 ,  0.23893555,  0.401075  ]], dtype=float32), hidden_states=None, attentions=None)

albert의 최종 accuracy는 0.302였습니다.

albert가 bert나 roberta보다 더 향산된 결과를 보인다는 블로그를 읽고 시도했고,
bert의 0.3보다 0.02 더 좋은 결과이긴 했지만, 유의미한 차이라는 느낌은 들지 않습니다.\
실행마다 값이 달리질 수도 있기 때문입니다.

그리고, epoch수를 늘려서 진행하고, pretraiend data를 바꿀시,\
다른 결과가 얼마든지 나올 수 있다는 생각이 들었습니다.

## 회고
---
이번 노드에서는 허긴페이스의 모델을 가져와서 학습하는 방법을 배웠습니다.\
이를 통해서 다양한 모델을 편하게 적용할 수 있게 됬습니다.

어느덧, 노드 과제의 마지막을 수행하게 되었습니다.\
짧은 시간동안 많은 내용이 빠르게 지나가면서, 코드를 분석하기보단,\
노드내에 있던 코드들을 복붙하기 바빴습니다.\
남은 시간들은 배운 것을 가다듬어야 하지 않을까 싶습니다.